# 3D printing with Jubilee
Let's start by initiating communication with the machine and define the gel extruder(s).

In [ ]:
%load_ext autoreload
%autoreload 2
from science_jubilee.Machine import Machine
from science_jubilee.tools.Tool import Tool
from science_jubilee.tools.SyringeExtruder import SyringeExtruder
from science_jubilee.tools.Camera import Camera

In [ ]:
m = Machine(address = "duet3.local")

The first thing you do is to home the machine, if you need to! Make sure the build plate is clear.

In [ ]:
m.home_all()

Homing could take a minute. Now, define and load tool(s).

In [ ]:
# Change your tool numbers to match your machine!
syringe_0 = SyringeExtruder(index = 2, name = "syr1", config = "10cc_syringe")
syringe_1 = SyringeExtruder(index = 3, name = "syr2", config = "10cc_syringe")
m.load_tool(syringe_0) # if you reinitiate a tool, use reload_tool instead of load_tool
m.load_tool(syringe_1)

3D printing fine structure is very sensitive to the z offset. Z offset changes with the nozzle you're using, the length of the syringe, etc. Now we need to zero the nozzle tip and update the z offset.

First, pick up the tool you'd like to calibrate.

In [ ]:
m.pickup_tool(syringe_0)

The z offset on startup should be slightly higher than the actual z offset, this is to avoid collisions with the bed. You can read more on it on the [Jubilee wiki](https://jubilee3d.com/index.php?title=Setting_Tool_Offsets).
Let's move to the current `Z = 0`.

In [ ]:
m.move_to(z = 0)

Bring the tip of the syringe extruder into contact with the print bed by gradually decreasing the z value. Don't overshoot or you'll crash into the syringe!

You can use the control panel in the DuetWebControl interface to do this if you like

In [ ]:
z_offset = -1.5 # slowly make this a bigger negative number, or use the duet web control panel
m.move_to(z = z_offset)

We'll update the tool's z offset so that the 0 position is where you currently moved to. Note that everytime the mainboard restarts, it reads the config on the Duet board, which has the more conservative tool offset. 


Instead of changing the offset permanently, we'll temporarily change it until we restart the machine again. So, we should go through this process each time we add a new syringe tip.

If you want to be able to take your print off the bed, be sure to put a glass plate down and calibrate to the height of the plate!

In [ ]:
starting_offset = m.tool_z_offsets[m.active_tool_index] # this is the default value for this tool
real_z_position = float(m.get_position()["Z"])          # this is our current z position after calibration
new_z_offset = starting_offset - real_z_position         # subtract the two to get the update z offset value

# Update the tool offset
m.set_tool_offset(tool_idx = m.active_tool_index, z = new_z_offset)

In [ ]:
# Our current z position should now be 0!
m.get_position()['Z']

Before loading your slicer-generated gcode file, it's a good practice to prime the nozzle - extrude a little bit of material so that the nozzle is filled and ready to print.

Noted that the parameter of `move_extrude` is the length of the filament, i.e. the actual plunger movement. 

In [ ]:
m.move_to(z = 25) # move the bed away if it's too close to the nozzle
syringe_0.move_extrude(e=1) # you may need to extrude many times. An empty tapered nozzle needs ~1.5 to fill up

Helper functions to load gcode and print gcode:

In [ ]:
# load the .gcode file and parse
def load_gcode(file_path):
    try:
        lines = []
        with open(file_path, 'r') as file:
            for line in file:
                lines.append(line.strip())
        return lines
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def print_gcode(gcode):
    for line in gcode:
        if len(line) > 0:
            if not line.startswith(';'):
                print(line)
                m.gcode(line)


In [ ]:
# Once you run this cell, the machine will start printing a small cylinder!
# Position your glass plate in the center of the bed if using
gcode = load_gcode("cylinder-20mm.gcode")
print_gcode(gcode)

In [ ]:
m.park_tool()

## Other 3D Printing Options!
With the syringe extruder tool, we can also directly specify simple toolpaths to print; this might be useful if you want to directly extrude in a particular location, or do other things of that nature

Note that this functionality is quite basic, especially compared to built-in slicer features! Expect to do a lot of tuning!

### Non-Planar cube
Here we'll print this test cube with one raised corner:

In [ ]:
# Pick up your syringe
m.pickup_tool(syringe_0)

In [ ]:
# Set up some printing parameters
# Change these if and as you'd like!
z = 0              # starting z value
layer_height = 0.2 
z_off = 0          # how much to offset one corner

start_x = 220      # x start position
start_y = 220      # y start position
side_length = 20   # side length of the cube

m.move_to(x=start_x, y=start_y, z=z)


# the move_extrude command will extrude gel from the current position to the specified position
# you can edit the extrusion multiplier if you'd like more/less gel extruded, or specify a speed to move at
for layer in range(40):
    syringe_0.move_extrude(x = start_x + side_length, y = start_y, z = z, multiplier = 1)
    syringe_0.move_extrude(x = start_x + side_length, y = start_y - side_length, z = z + z_off, multiplier = 1)
    syringe_0.move_extrude(x = start_x, y = start_y - side_length, z = z + z_off, multiplier = 1)
    syringe_0.move_extrude(x = start_x, y =start_y, z = z, multiplier = 1)
    z += layer_height
    z_off += 0.1

### Printing + Taking Pics!
Here we switch between the syringe and camera tool to take pictures of each layer of the print! It's hard to do well. 

In [ ]:
# Add your camera tool
camera = Camera(1, "cam")
m.load_tool(camera)

In [ ]:
%matplotlib inline
# ^ this will show pictures taken inline in the notebook

z = 0.1
layer_height = 0.2
theta = 0
center_x = 220
center_y = 220
center = [center_x, center_y]
radius = 10
theta_step = 2 * math.pi / 50
phase = 0
camera_focus_height = 30 # I manually focused the camera at z = 30

while z < 20:
    m.pickup_tool(syringe_0)
    
    # wipe_tower will make a cube at the location specified to help get the gel flowing
    syringe_0.wipe_tower(x = center_x, y = center_y - 50, z=z)
    
    # extrude a circle
    while theta < 2 * math.pi:
        point = [radius * math.cos(theta) + center_x, radius * math.sin(theta) + center_y]
        if theta==0:
            m.move_to(x = point[0], y = point[1], z = z, s=5000)
        else:
            syringe_0.move_extrude(x = point[0], y = point[1], z = z, multiplier = 1)
        theta += theta_step
    z += layer_height
    theta = 0
    radius += 0.5
    
    # now take an image
    m.pickup_tool(camera)
    m.move_to(z=camera_focus_height+z) # focus height above top of print
    m.move_to(x=center_x, y=center_y)
    frame = camera.get_frame()
    camera.show_frame(frame, save=True, save_path=f'/home/pi/syringe_z{z}.jpg') # save a picture
    m.park_tool() # put the camera back
    m.move_to(z=z) # bring the bed back up so that we can wipe the nozzle on syringe pickup